In [1]:
import sys
from pathlib import Path

ROOT_DIR = Path().resolve().parent
sys.path.append(str(ROOT_DIR))

In [2]:
from src.loader_dataset import load_names, get_caption_by_image_name
from src.embeddings import generate_text_embeddings, generate_image_embeddings, combine_img_embeddings_text_embeddings
from src.vector import build_vector_index, search_similar_vectors, save_index, load_index

[nltk_data] Downloading package stopwords to C:\Users\Mauricio
[nltk_data]     Morales\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\Mauricio Morales\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
paths = ['../data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv','../data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv','../data/1429_1.csv' ]
img_dir = '../data/img'

df = load_names(paths, img_dir)
df

C:\Users\Mauricio Morales\Documents\Universidad\2025b\RI\RI_ICCD753_PROJECT_BIM_2\src\loader_dataset.py:26: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  [pd.read_csv(p) for p in csv_paths],


,image_path,caption,brand,primaryCategories,categories
0,../data/img/all new echo nd generation with im...,all new echo nd generation with improved sound...,Amazon,"Electronics,Furniture","Amazon Echo,Smart Speakers,Electronics,Digital..."
1,../data/img/all new echo nd generation with im...,all new echo nd generation with improved sound...,Amazon,"Electronics,Furniture","Amazon Echo,Smart Speakers,Electronics,Digital..."
2,../data/img/all new echo nd generation with im...,all new echo nd generation with improved sound...,Amazon,"Electronics,Furniture","Amazon Echo,Smart Speakers,Electronics,Digital..."
3,../data/img/all new echo nd generation with im...,all new echo nd generation with improved sound...,Amazon,"Electronics,Furniture","Amazon Echo,Smart Speakers,Electronics,Digital..."
4,../data/img/all new echo nd generation with im...,all new echo nd generation with improved sound...,Amazon,"Electronics,Furniture","Amazon Echo,Smart Speakers,Electronics,Digital..."
...,...,...,...,...,...
372,../data/img/two door top load pet kennel trave...,two door top load pet kennel travel crate dog ...,Amazonbasics,Animals & Pet Supplies,"Carriers & Totes,Hard-Sided Carriers,Crates Ke..."
373,../data/img/two door top load pet kennel trave...,two door top load pet kennel travel crate dog ...,Amazonbasics,Animals & Pet Supplies,"Carriers & Totes,Hard-Sided Carriers,Crates Ke..."
374,../data/img/two door top load pet kennel trave...,two door top load pet kennel travel crate dog ...,Amazonbasics,Animals & Pet Supplies,"Carriers & Totes,Hard-Sided Carriers,Crates Ke..."
375,../data/img/two door top load pet kennel trave...,two door top load pet kennel travel crate dog ...,Amazonbasics,Animals & Pet Supplies,"Carriers & Totes,Hard-Sided Carriers,Crates Ke..."


In [4]:
embedding = generate_text_embeddings(df['caption'].to_list(), device = 'cuda')
print(type(embedding))
embedding

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Batches: 100%|██████████| 12/12 [00:00<00:00, 30.41it/s]

<class 'numpy.ndarray'>


array([[ 0.08888689, -0.00419313, -0.280248  , ...,  0.32537943,
        -0.20042215,  0.19829024],
       [ 0.08888689, -0.00419313, -0.280248  , ...,  0.32537943,
        -0.20042215,  0.19829024],
       [ 0.08888689, -0.00419313, -0.280248  , ...,  0.32537943,
        -0.20042215,  0.19829024],
       ...,
       [-0.23860979, -0.47660258, -0.75309044, ...,  0.10924853,
        -0.14828525, -0.05944708],
       [-0.23860979, -0.47660258, -0.75309044, ...,  0.10924853,
        -0.14828525, -0.05944708],
       [-0.23860979, -0.47660258, -0.75309044, ...,  0.10924853,
        -0.14828525, -0.05944708]], shape=(377, 512), dtype=float32)

In [5]:
embedding_img, paths = generate_image_embeddings('../data/img/', device='cuda')
print(type(embedding_img))
embedding_img

Batches: 100%|██████████| 12/12 [00:01<00:00,  6.53it/s]

<class 'numpy.ndarray'>


array([[ 0.13103196,  0.16740093,  0.1666284 , ...,  1.2741207 ,
        -0.59914815, -0.18903011],
       [ 0.36413854, -0.6005052 , -0.27918732, ..., -0.0407021 ,
         0.20372856, -0.10125706],
       [ 0.03712692,  0.0906371 , -0.23609671, ...,  0.2547573 ,
        -0.26872647,  0.02006569],
       ...,
       [ 0.15266132, -0.15757117, -0.10745257, ...,  1.6013615 ,
         0.12703656,  0.14768949],
       [ 0.19886822,  0.0869606 , -0.01814518, ...,  1.2301664 ,
         0.20184012,  0.30138808],
       [ 0.28987283, -0.28700414, -0.2446162 , ...,  1.4398332 ,
         0.10557139, -0.03022031]], shape=(377, 512), dtype=float32)

In [11]:
import numpy as np
emb = np.array(combine_img_embeddings_text_embeddings(embedding, embedding_img))

In [ ]:
index = build_vector_index(emb)
query = ['fire_stick']
query_emb = generate_text_embeddings(query)
D, I = search_similar_vectors(query_emb, index, K=10)
results_df = df.iloc[I[0]]

top_unique = results_df.drop_duplicates(subset=['caption']).head(10)

top_unique[['caption', 'categories', 'primaryCategories']]

Batches: 100%|██████████| 1/1 [00:00<00:00, 75.03it/s]

                                               caption  \
266                  certified refurbished amazon echo   
96   amazon fire tv gaming edition streaming media ...   
50                                amazon echo ai white   
1    all new echo nd generation with improved sound...   

                                            categories      primaryCategories  
266  Consumer Electronics,Amazon Echo,Smart Speaker...            Electronics  
96   Amazon SMP,Electronics Features,Kids & Family,...            Electronics  
50   Stereos,Remote Controls,Audio Docks & Mini Spe...            Electronics  
1    Amazon Echo,Smart Speakers,Electronics,Digital...  Electronics,Furniture  
